### Exercice 1: Calculer la moyenne de la puissance administrative (puiss_admin_98) pour chaque marque de voiture (lib_mrq).



In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()
df = spark.read.csv('car.csv', header=True, inferSchema=True, sep=';')

In [38]:
df.groupby(df.lib_mrq)\
  .agg(
    avg(df.puiss_admin_98).alias('avg_puiss_admin_98')
  )\
  .show()

+-----------+------------------+
|    lib_mrq|avg_puiss_admin_98|
+-----------+------------------+
|   MERCEDES|10.988514632799559|
|LAMBORGHINI|53.388888888888886|
|    HYUNDAI|6.9411764705882355|
|      DACIA| 5.137931034482759|
|       FIAT|          6.296875|
|    CITROEN| 6.468181818181818|
|    BENTLEY| 48.15384615384615|
|       LADA|               6.0|
|   INFINITI|           17.6875|
|       AUDI|12.919770773638968|
|       FORD| 7.406040268456376|
|    FERRARI|             52.44|
|  CHEVROLET| 14.43421052631579|
|     JAGUAR|26.666666666666668|
| ALFA-ROMEO| 7.797814207650274|
|      LEXUS|18.833333333333332|
|       JEEP| 16.43243243243243|
|      MAZDA| 8.073170731707316|
|        BMW|14.362850971922246|
|   MASERATI|29.083333333333332|
+-----------+------------------+
only showing top 20 rows



### Exercice 2: Trouver la voiture ayant la consommation mixte (conso_mixte) la plus basse.



In [63]:
min_conso_mixte = df.agg(min(df.conso_mixte)).collect()[0][0]
df.filter(df.conso_mixte == min_conso_mixte)\
  .collect()[0].asDict()

{'lib_mrq': 'BMW',
 'lib_mod_doss': 'SERIE I',
 'lib_mod': 'I3',
 'dscom': "i3 avec Prolongateur d'autonomie",
 'cnit': 'M10BM2VP0000008',
 'tvv': 'BMWi-11Z416A040000',
 'cod_cbr': 'EE',
 'hybride': 'oui',
 'puiss_admin_98': 3,
 'puiss_max': '28',
 'typ_boite_nb_rapp': 'V 0',
 'conso_urb': None,
 'conso_exurb': None,
 'conso_mixte': '0,600000024',
 'co2': 13,
 'co_typ_1': '0,052999999',
 'hc': '0,004',
 'nox': '0,002',
 'hcnox': None,
 'ptcl': None,
 'masse_ordma_min': 1390,
 'masse_ordma_max': 1390,
 'champ_v9': '715/2007*195/2013EURO6',
 'date_maj': 'mars-14',
 'Carrosserie': 'BERLINE',
 'gamme': 'INFERIEURE',
 '_c26': None,
 '_c27': None,
 '_c28': None,
 '_c29': None}

### Exercice 3: Filtrer les voitures dont la puissance maximale (puiss_max) est supérieure à 200 chevaux.



In [83]:
df.filter(
    df.puiss_max > 200
  ).count()

1781

### Exercice 4: Calculer le nombre de voitures hybrides par carrosserie.



In [116]:
df.filter(df.hybride == 'oui')\
  .groupby(df.Carrosserie, df.hybride)\
  .count()\
  .show()

+-------------------+-------+-----+
|        Carrosserie|hybride|count|
+-------------------+-------+-----+
|          MINISPACE|    oui|    1|
|              BREAK|    oui|   11|
|TS TERRAINS/CHEMINS|    oui|   47|
|              COUPE|    oui|    5|
|            BERLINE|    oui|  325|
+-------------------+-------+-----+



### Exercice 5: Calculer le pourcentage de voitures émettant moins de 120 g/km de CO<sub>2</sub> (co2) par gamme de voiture.



In [139]:
df.groupby(df.gamme).count()\
  .withColumnRenamed('count', 'nb_total')\
  .join(
    df.filter(df.co2 < 120)\
      .groupby(df.gamme).count()\
      .withColumnRenamed('count', 'nb_cars_co2_lt_120'),
    on='gamme'
  ).withColumn('percentage_of_total', 
              round(
                  col('nb_cars_co2_lt_120') / col('nb_total') * 100,
                  2
              )
  )\
  .show()

+----------+--------+------------------+-------------------+
|     gamme|nb_total|nb_cars_co2_lt_120|percentage_of_total|
+----------+--------+------------------+-------------------+
| MOY-INFER|   33438|               762|               2.28|
|SUPERIEURE|    1513|               156|              10.31|
|ECONOMIQUE|     269|               171|              63.57|
|      LUXE|    3597|               112|               3.11|
|INFERIEURE|    1073|               473|              44.08|
| MOY-SUPER|   15098|               148|               0.98|
+----------+--------+------------------+-------------------+



### Exercice 6: Trouver la carrosserie la plus courante parmi les voitures de chaque gamme.



In [196]:
df.groupby('gamme', 'Carrosserie')\
  .agg(count('*').alias('count'))\
  .sort(col('gamme').asc(),
        col('Carrosserie').asc())\
  .groupby('gamme', 'Carrosserie')\
  .agg({'count': 'max'})\
  .show()

+----------+-------------------+----------+
|     gamme|        Carrosserie|max(count)|
+----------+-------------------+----------+
|SUPERIEURE|              BREAK|       345|
|ECONOMIQUE|          MINISPACE|        48|
| MOY-SUPER|TS TERRAINS/CHEMINS|       252|
|      LUXE|            BERLINE|      1615|
| MOY-INFER|  MONOSPACE COMPACT|       319|
| MOY-INFER|            MINIBUS|     31772|
|SUPERIEURE|          CABRIOLET|       100|
| MOY-INFER|              BREAK|       177|
| MOY-INFER|            BERLINE|      1015|
|INFERIEURE|            BERLINE|       542|
|SUPERIEURE|TS TERRAINS/CHEMINS|        65|
|INFERIEURE|          MINISPACE|        97|
|INFERIEURE|         COMBISPACE|       235|
| MOY-SUPER|            BERLINE|       266|
|SUPERIEURE|            BERLINE|       799|
| MOY-INFER|          CABRIOLET|        65|
|INFERIEURE|TS TERRAINS/CHEMINS|       122|
| MOY-SUPER|              COUPE|        83|
|      LUXE|              COUPE|       599|
| MOY-INFER|TS TERRAINS/CHEMINS|

### Exercice 7: Calculer la différence entre la puissance maximale (puiss_max) et la puissance administrative (puiss_admin_98) pour chaque voiture.



### Exercice 8: Trouver les marques de voiture (lib_mrq) dont la consommation urbaine (conso_urb) est inférieure à la consommation mixte (conso_mixte).



### Exercice 9: Calculer la moyenne des émissions de CO<sub>2</sub> (co2) par année de mise à jour (annee_maj).



### Exercice 10: Trouver les voitures dont la masse ordma_max est supérieure à la moyenne de la masse ordma_max de toutes les voitures.